# Online Reinforcement Learning

[Course Websites](https://www.deeplearning.ai/short-courses/post-training-of-llms/)

## Introduction

- Offline RL: The model learns from a purely pre-collected prompt-response(-reward) tuple. It is like a huge defined table for action space and reward functions. No fresh responses generated during the training process.
    - It is more like a detective.
    - The action space and reward functions are all finite.
    - The ultimate goal for offline RL is to learn optimized strategies based on stable historical data.

- Online RL:
    - We will interact with environment!
    - The model learns by generating new responses in real time, and update its weights.

## Online RL methods

Prompts $\to$ Current Models $\to$ (Prompts, Answers) $\to$ (Prompts, Answers, Rewards) (Given a fixed reward function.)

- TRPO (Trust Region Policy Optimization)
- PPO (Proximal Policy Optimization)
    - Using "Proximal" instead of trusted optimization.
- GRPO 

[PPO & GRPO](https://builder.aws.com/content/2rJrpj6m2eh591fjMcRZ3ushpB7/deep-dive-into-group-relative-policy-optimization-grpo)

## Preliminaries

### 策略评估恒等式

**策略评估恒等式**。它揭示了新策略的期望回报与旧策略的期望回报之间的关系。

期望回报 Expected Return

$$\eta(\pi_\theta) = \mathbb{E}_{s \sim \rho_{\pi_\theta}, a \sim \pi_\theta} \left[ \sum_{t=0}^\infty \gamma^t R(s_t, a_t) \right]$$

* **$\eta(\pi_\theta)$**: 表示在**策略 $\pi_\theta$** 下的**期望累积回报**（或称价值）。这是我们想要优化的最终目标。
* **$\mathbb{E}_{s \sim \rho_{\pi_\theta}, a \sim \pi_\theta}$**: 表示期望的来源。我们对所有可能的**状态-动作轨迹**取期望。
    * **$s \sim \rho_{\pi_\theta}$**: 状态 $s$ 来自于在策略 $\pi_\theta$ 下的**状态访问分布**。这表示智能体在执行策略 $\pi_\theta$ 时，访问到各个状态的概率。
    * **$a \sim \pi_\theta$**: 动作 $a$ 来自于策略 $\pi_\theta$。
* **$\sum_{t=0}^\infty \gamma^t R(s_t, a_t)$**: 这部分是**折扣累积回报**，也叫**回报**（Return）。$R(s_t, a_t)$ 是在时间步 $t$ 时采取动作 $a_t$ 获得的即时奖励，$\gamma$ 是折扣因子，用来衡量未来奖励的重要性。

从定义出发经过严格的数学推导，可以得到下面的更复杂的**策略评估恒等式**，它将新策略的回报与旧策略的回报以及两者之间的优势函数联系起来：

$$\eta(\pi_\theta) = \eta(\pi_{\theta_{old}}) + \mathbb{E}_{\tau \sim \pi_\theta} \left[ \sum_{t=0}^\infty \gamma^t A_{\pi_{\theta_{old}}}(s_t, a_t) \right]$$

这个公式的推导过程比较复杂，但其核心思想是基于**状态价值函数**和**优势函数**的定义。

**推导过程：**

1.  **从状态价值函数开始**：一个策略的价值，可以分解为在每个状态下的价值。
    $$\eta(\pi_\theta) = \mathbb{E}_{s_0 \sim \rho_0} [V^{\pi_\theta}(s_0)]$$
    其中 $\rho_0$ 是初始状态分布。

2.  **利用贝尔曼方程**：状态价值函数 $V^\pi(s)$ 和状态-动作价值函数 $Q^\pi(s, a)$ 满足贝尔曼方程。一个关键的恒等式是：
    $$V^{\pi}(s) = \mathbb{E}_{a \sim \pi} [Q^\pi(s, a)] = \mathbb{E}_{a \sim \pi} [A^\pi(s, a) + V^\pi(s)]$$
    这可以推导出：
    $$V^{\pi}(s) = \mathbb{E}_{a \sim \pi}[A^\pi(s,a)] + V^\pi(s)$$
    这个式子看似无用，但如果把 $V^\pi(s)$ 替换为**旧策略的价值** $V^{\pi_{old}}(s)$，并用**新策略** $\pi_\theta$ 的动作来求期望，就会有新的含义：
    $$V^{\pi_\theta}(s) = \mathbb{E}_{a \sim \pi_\theta} [Q^{\pi_{old}}(s,a)]$$
    根据 $Q(s, a) = V(s) + A(s, a)$，可以得到：
    $$V^{\pi_\theta}(s) = \mathbb{E}_{a \sim \pi_\theta} [V^{\pi_{old}}(s) + A^{\pi_{old}}(s,a)] = V^{\pi_{old}}(s) + \mathbb{E}_{a \sim \pi_\theta} [A^{\pi_{old}}(s,a)]$$

3.  **对所有状态进行累加**：这个关系式在每个状态都成立。我们可以通过将它扩展到整个马尔可夫决策过程，来得到一个更通用的**累积回报恒等式**。这是通过**马尔可夫链**的性质得到的。最终推导结果正是你图片中的第二个公式。

* **$\eta(\pi_{\theta_{old}})$**: 这是**旧策略 $\pi_{\theta_{old}}$** 的期望回报，一个基准值。
* **$\mathbb{E}_{\tau \sim \pi_\theta}$**: 表示期望的来源是**新策略** $\pi_\theta$ 产生的**轨迹** $\tau$。
* **$\sum_{t=0}^\infty \gamma^t A_{\pi_{\theta_{old}}}(s_t, a_t)$**: 这是**优势函数的累积和**。
    * **$s_t, a_t$**: 这对状态-动作对是在**新策略** $\pi_\theta$ 下被访问或执行的。
    * **$A_{\pi_{\theta_{old}}}(s_t, a_t)$**: 这表明我们评估的优势函数是**基于旧策略**的，它衡量了**新策略**在访问的状态上所采取的动作，相对于**旧策略**的平均表现如何。

第二个公式表明，新策略的回报等于旧策略的回报**加上**在新策略下，**沿途所遇到的所有状态和动作的优势函数的累积期望**。换句话说，要提升策略，你只需要在新策略下，更多地执行那些在旧策略看来是**好**的动作（**$A>0$**），并避免那些**坏**的动作（**$A<0$**）。这就是 TRPO 和 PPO 算法的核心思想来源。

### 重要性采样

我们想要优化的目标是新策略的期望回报 $\eta(\pi_\theta)$。我们已经知道一个重要的关系：

$$\eta(\pi_\theta) = \eta(\pi_{\theta_{old}}) + \mathbb{E}_{\tau \sim \pi_\theta} \left[ \sum_{t=0}^\infty \gamma^t A_{\pi_{\theta_{old}}}(s_t, a_t) \right]$$

问题出在右边的期望项 $\mathbb{E}_{\tau \sim \pi_\theta}[\dots]$。这个期望是**在新策略 $\pi_\theta$ 的轨迹分布下**求的。但在训练过程中，我们只有**旧策略 $\pi_{\theta_{old}}$** 产生的数据。我们必须找到一个方法，**用旧策略的数据来近似计算新策略的期望**

重要性采样的基本思想是：如果我想要计算一个函数在某个分布下的期望，但从这个分布中采样很困难，我可以转而从另一个更容易采样的分布中获取样本，然后通过一个**权重**来修正。

数学上，它的公式是这样的：
$$\mathbb{E}_{x \sim P} [f(x)] = \mathbb{E}_{x \sim Q} \left[ \frac{P(x)}{Q(x)} f(x) \right]$$

这里：
* $P$ 是我们想要的**目标分布**（即新策略 $\pi_\theta$）。
* $Q$ 是我们实际采样的**行为分布**（即旧策略 $\pi_{\theta_{old}}$）。
* $\frac{P(x)}{Q(x)}$ 是**修正权重**，也就是我们说的**重要性采样比值**。

现在，我们把这个思想应用到我们的期望项上：

* **目标分布 $P$**: 是新策略产生的轨迹分布 $\tau \sim \pi_\theta$。
* **行为分布 $Q$**: 是旧策略产生的轨迹分布 $\tau \sim \pi_{\theta_{old}}$。
* **函数 $f(\tau)$**: 是轨迹回报的累积和 $\sum_{t=0}^\infty \gamma^t A_{\pi_{\theta_{old}}}(s_t, a_t)$。

将这些代入重要性采样的公式，我们会得到：

$$\mathbb{E}_{\tau \sim \pi_\theta} \left[ \sum_{t=0}^\infty \gamma^t A_{\pi_{\theta}}(s_t, a_t) \right] = \mathbb{E}_{\tau \sim \pi_{\theta_{old}}} \left[ \frac{P(\tau)}{Q(\tau)} \cdot \left(\sum_{t=0}^\infty \gamma^t A_{\pi_{\theta_{old}}}(s_t, a_t)\right) \right]$$

这里的 $\frac{P(\tau)}{Q(\tau)}$ 是轨迹的概率比值，它等于沿途所有状态-动作对的概率比值的乘积。

$$\frac{P(\tau)}{Q(\tau)} = \prod_{t=0}^\infty \frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_{old}}(a_t|s_t)}$$

这个连乘积的计算非常复杂，因此在实际计算中采用**蒙特卡洛模拟**的方式，只模拟单步的优化表现。

### TRPO (Trust Region Policy Optimization)

https://arxiv.org/pdf/1502.05477

TRPO的核心思想是：在策略优化过程中，我们希望**策略的性能单调提升**，并且避免因**更新步长过大而导致性能崩溃**。它通过信任区域来限制新策略与旧策略之间的差异，确保更新在安全的范围内进行。

> 这也是强化学习最基本的思想之一：The tradeoff between exploration and exploitation.

策略梯度方法的最终目标是最大化期望累积回报 $J(\pi_\theta)$。TRPO的优化目标源于一个理论上的下界。

经过近似推导（忽略状态分布的变化），可以得到一个替代优势（Surrogate Advantage）： $$L_{\pi_{\theta_{old}}}(\pi_\theta) = \eta(\pi_{\theta_{old}}) + \mathbb{E}_{s \sim \rho_{\pi_{\theta_{old}}}, a \sim \pi_{\theta_{old}}} \left[ \frac{\pi_\theta(a|s)}{\pi_{\theta_{old}}(a|s)} A_{\pi_{\theta_{old}}}(s, a) \right]$$

最大化 $L$ 理论上可以保证提升真实的 $\eta$。因此，TRPO的目标函数是最大化这个替代优势的第二部分： $$J_{TRPO}(\theta) = \mathbb{E}_{t} \left[ \frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_{old}}(a_t|s_t)} \hat{A}_t \right]$$

然而，上述近似只有在 $\pi_\theta$ 和 $\pi_{\theta_{old}}$ 足够接近时才有效。TRPO使用**KL散度**作为两个策略分布差异的度量，并将这个差异约束在一个信任区域 $\delta$ 内。

$$\mathbb{E}_{s\sim\pi_{\theta_{old}}}[KL[\pi_{\theta_{old}}(\cdot|s),\pi_\theta(\cdot|s)]]\leq\delta$$


最终，TRPO的优化问题是一个带约束的优化问题： 

$$\max_{\theta}\quad\mathbb{E}_{s,a\sim\pi_{\theta_{odd}}}\left[\frac{\pi_{\theta}(a|s)}{\pi_{\theta_{odd}}(a|s)}\hat{A}(s,a)\right]$$

$$\mathrm{subject~to}\quad\mathbb{E}_{s\sim\pi_{\rho_{odd}}}[KL[\pi_{\theta_{odd}}(\cdot|s),\pi_{\theta}(\cdot|s)]]\leq\delta$$

直接求解这个带约束的问题计算量很大。TRPO使用共轭梯度法和线搜索来近似求解。

- 首先，将目标函数做一阶近似（即梯度），将约束做二阶近似（因为KL散度的一阶导为0，二阶导是Fisher信息矩阵 $F$）。
- 这样问题转化为一个二次规划问题，其解的方向为 $\theta - \theta_{old} \propto F^{-1} \nabla_\theta J$，即自然梯度的方向。
- 计算出更新方向后，再通过线搜索找到一个合适的步长，确保KL散度约束被满足的同时，目标函数确实有所提升。

我们想找到一个更新方向 $\Delta\theta = \theta - \theta_{old}$，来最大化目标函数。在参数 $\theta_{old}$ 的附近，我们可以对目标函数进行**一阶泰勒展开**。

最大化目标函数就近似于**最大化梯度方向上的更新步长** $g^T(\theta - \theta_{old})$。


KL 散度函数在两个分布相同时，值为零，且其**一阶导数也为零**。因此，为了得到一个非零的近似，我们必须使用**二阶泰勒展开**。

* **泰勒展开**：$f(x) \approx f(a) + f'(a)(x-a) + \frac{1}{2}f''(a)(x-a)^2$
* **TRPO 约束**：我们对 KL 散度在 $\theta = \theta_{old}$ 处进行展开。

$$KL[\pi_{\theta_{old}}(\cdot|s), \pi_\theta(\cdot|s)] \approx \frac{1}{2}(\theta - \theta_{old})^T F(s) (\theta - \theta_{old})$$
其中 **$F(s)$ 是 Fisher 信息矩阵**，它衡量了在状态 $s$ 下，策略参数的变化对策略分布的影响。

对所有状态取平均，我们得到约束的二阶近似：
$$\mathbb{E}_{s \sim \pi_{\theta_{old}}} [KL[\pi_{\theta_{old}}(\cdot|s), \pi_\theta(\cdot|s)]] \approx \frac{1}{2}(\theta - \theta_{old})^T F (\theta - \theta_{old})$$
其中 $F = \mathbb{E}_{s \sim \pi_{\theta_{old}}}[F(s)]$ 是**平均 Fisher 信息矩阵**。

通过这些近似，原始的 TRPO 优化问题被转化为了一个更简单的**二次规划**问题：

$$\begin{align} \underset{\Delta\theta}{\max} \quad & g^T \Delta\theta \\ \text{subject to} \quad & \frac{1}{2}\Delta\theta^T F \Delta\theta \leq \delta \end{align}$$

这个问题的解有一个非常著名的名称：**自然梯度**。它是一个更优化的梯度方向，它**考虑了参数空间中的几何结构**。它确保了更新方向不仅仅在参数上是“陡峭的”，在策略分布上也是“有意义的”。

这个问题的解析解是：

$$\Delta\theta \propto F^{-1}g$$

这就是**自然梯度的方向**。TRPO 的巧妙之处在于，它不需要显式地计算 Fisher 矩阵的逆 $F^{-1}$（计算量巨大），而是使用**共轭梯度法**来高效地求解一个线性方程，从而得到自然梯度的方向。

### PPO

PPO（**Proximal Policy Optimization**）是一种强化学习算法，它旨在解决传统策略梯度方法训练不稳定、容易崩溃的问题。PPO 的核心思想是**用一种简单但有效的方式，来近似实现 TRPO 的稳健性**。

与 TRPO 类似，PPO 同样使用一个替代目标函数来近似策略的提升。这个函数基于**重要性采样**，用旧策略的数据来评估新策略的性能。

PPO 的目标函数和 TRPO 保持相同：

$$L(\theta) = \mathbb{E}_{t} \left[ \frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_{old}}(a_t|s_t)} \hat{A}_t \right]$$

但是 PPO **不使用复杂的 KL 散度约束**，而是通过**裁剪**重要性采样比值 $r_t(\theta)$ 来限制策略的更新幅度。

PPO 的裁剪目标函数是：

$$L^{CLIP}(\theta) = \mathbb{E}_{t} \left[ \min\left(r_t(\theta)\hat{A}_t, \text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon)\hat{A}_t\right) \right]$$

让我们分解一下这个公式：

* **$r_t(\theta)\hat{A}_t$**：原始的策略梯度目标，也就是不加限制的更新。
* **$\text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon)$**：将 $r_t(\theta)$ 裁剪到 $[1-\epsilon, 1+\epsilon]$ 这个区间内。
    * $\epsilon$ 是一个超参数，通常取值很小（如 0.1 或 0.2）。
    * 这意味着，如果新策略选择某个动作的概率比旧策略**大得多**（$r_t > 1+\epsilon$），它会被限制在 $1+\epsilon$。
    * 如果新策略选择某个动作的概率比旧策略**小得多**（$r_t < 1-\epsilon$），它会被限制在 $1-\epsilon$。

这样的裁剪可以达到和 TRPO 中的区域约束很像，都是惩罚模型过度的更新权重，一次迈步迈太大而导致表现不稳定，性能迅速下降的现象。同时，这个相比于 TRPO 的二阶优化约束需要更少的计算量，但是达到了相同的惩罚效果。

PPO 采用标准的**随机梯度下降（SGD）**来优化裁剪后的目标函数，并结合**多轮小批量训练**。
> 因为这个函数是可微分的。

* **数据收集**：使用当前策略与环境互动，收集一批数据（如 2048 个时间步）。
* **多轮训练**：使用这批数据，对策略网络和价值网络进行多次迭代更新（例如，进行 10 轮训练）。每轮训练都将数据打乱，分成小批量进行梯度下降。

### GRPO

![GRPO](https://assets.community.aws/a/2ra6RfOUylhUM8M3eDEMfDlB3l7/Scre.webp?imgSize=1698x894)